# 데이터 준비

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [4]:
input_data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

train_input, test_input, train_target, test_target = train_test_split(
    input_data, target, test_size=0.2, random_state=42
)

---
# 앙상블 학습(Ensemble Learning)
- 정형 데이터: 수치형 데이터 등 특정 구조로 저장할 수 있는 데이터. CSV, 데이터베이스, 엑셀 등에 저장하기 쉬운 데이터
- 비정형 데이터: 정형 데이터와 다른 형태의 데이터(이미지, 음악, 텍스트 데이터, ... 등)
    - 정형 데이터를 다루는데 가장 좋은 성능을 나타내는 알고리즘이 앙상블 학습임.
    - 반면 비정형 데이터는 규칙성을 찾기 어려워, 전통적인 머신러닝 방법으로는 모델을 만들기 까다로움 -> 신경망 알고리즘을 통해 해결

---
# 랜덤 포레스트
- 결정 트리를 여러 개 만들어서 숲을 만드는 방법 -> 각 결정 트리의 예측을 활용하여 최종적인 예측을 만들어냄
- 랜덤 포레스트는 각 트리를 훈련시키기 위한 데이터를 만드는 방법이 독특한데, 이게 중요함
    - e.g. 전체 데이터셋이 1000개이고, 트리를 훈련시키기 위해 100개의 데이터를 뽑는다고 할 때, 1000개 중 100개를 복원추출을 통해 뽑아냄
    - 이렇게 복원 추출 방식으로 뽑아낸 데이터셋을 부트스트랩 샘플(bootstrap sample)이라고 함.
- 또한, 랜덤 포레스트는 전체 피쳐들의 개수의 제곱근만큼의 피쳐를 선택하고, 그중에서 최선의 분할을 찾는다.

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9973541965122431
0.8903229806766861


피쳐의 중요도를 출력해보자.

In [6]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23183515 0.50059756 0.26756729]


👉 처음 결정 트리를 통해 출력해본 피쳐의 중요도는 약 `[0.123, 0.87, 0.008]`정도였는데, 지금이랑 비교를 해보자.
        
- 랜덤 포레스트는 피쳐의 일부를 랜덤하게 선택해서 결정 트리를 훈련하기 떄문에, 그 결과 하나의 피쳐에만 과하게 집중하지 않고, 좀 더 많고 다양한 피쳐가 학습에 기여할 기회를 얻게 되는것과 같다. 
- 이를 통해 오버피팅을 줄이고 모델의 일반화 성능을 높일 수 있게됨

👉 `RandomForestClassifier`에는 자체적으로 모델을 평가하는 점수를 얻을 수 있는 기능이 있음.
- 랜덤 포레스트는 훈련셋에서 복원 추출을 통해 부트스트랩 샘플을 만들고, 결정 트리를 훈련한다고 했음
    - 이 때 각 결정트리마다 샘플에 포함되지 않는 샘플이 있는데(OOB 샘플이라고 함), 이를 이용해서 Validation Set처럼 활용하면 좋음. 이를 통해 해당 결정 트리를 '평가'할 수 있게됨
    - 이 값을 얻으려면 `oob_score`매개변수를 True로 하면 됨(기본값은 False)

In [7]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8945545507023283


- 결정 트리를 100그루 만들어도, 최종 스코어는 전체 데이터에 대한 한 개의 OOB스코어만 계산한다고 함(구체적으로는 잘 모르겠음)

---
# 엑스트라 트리(Extra Tree)
- 엑스트라 트리는 랜덤 포레스트와 매우 유사하게 동작
    - 랜덤 포레스트의 매개변수들을 거의 대부분 동일하게 지원함
    - 전체 피쳐 중 일부 피쳐를 랜덤하게 선택하여 노드를 분할하는 점도 동일
- 다만 차이점은, 엑스트라 트리는 부트스트랩 샘플을 사용하지 않는다는 점. 즉 각 결정 트리를 만들 때, 전체 훈련 세트를 사용함.
    - 그 대신 노드를 분할할 때, 가장 좋은 분할을 찾는게 아니라 무작위로 분할함
    - 2장에서 `DecisionTreeClassifier`클래스의 `splitter`매개변수를 `random`으로 설정했었는데, 엑스트라 트리가 사용하는 결정 트리가 `splitter=True`인 결정 트리이다.
- (참고)엑스트라 트리의 회귀 버전은 `ExtraTreesRegressor`클래스이다.

In [8]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, 
                        return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']))
print(np.mean(scores['test_score']))

0.9974503966084433
0.8887848893166506


👉 본 예제에서는 피쳐 개수가 많이 않아 랜덤 포레스트와 엑스트라 트리의 성능차이가 크게 안남

- 보통 엑스트라 트리가 무작위성이 더 크기 때문에, 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야한다.
- 다만, 엑스트라 트리의 장점은 노드를 랜덤하게 분할하기 때문에 계산 속도가 랜덤 포레스트보다 빠르다.
    - 결정 트리에서 시간을 많이 쓴느 부분은 최적의 분할을 찾는데에 많이 쓰는데, 특히 피쳐 개수가 많으면 유독 심해진다. 이걸 무작위로 나누게되면 트리를 훨씬 빨리 구성할 수 있게된다.

In [9]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


---
# 그래디언트 부스팅(Gradient Boosting)

---
# 히스토그램 기반 그래디언트 부스팅(Histogram-based Gradient Boosting)